<img src="https://raw.githubusercontent.com/rbizoi/PythonFormation/main/images/e-brasil.png" width="850">

https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce

# Les imports et initialisations des variables

In [1]:
from datetime import datetime
import pandas as pd, numpy as np, os, warnings, seaborn as sns, pickle, re, unicodedata
from datetime import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

font1 = fm.FontProperties(size=20)
font2 = fm.FontProperties(size=24)

%matplotlib inline

if int(str(sns.__version__).split('.')[1]) > 8 : 
    plt.style.use('seaborn-v0_8-darkgrid')
else:
    plt.style.use('seaborn-darkgrid')
    
sns.set(font_scale=2)
warnings.filterwarnings(action="ignore")

## Liste fichiers de données 

In [2]:
!dir ..\donnees\ebrasil
os.chdir("C:/Users/etien/Documents/CCI/_HUB/cours/2024-2025/big_data/_git/donnees")

Le fichier spécifié est introuvable.


## Changement de répertoire

In [3]:
os.chdir(r"../donnees")

# DataFrame $products$ 

In [4]:
products = pd.read_csv(os.path.join('ebrasil', 'olist_products_dataset.csv'))
products_translated = pd.read_csv(os.path.join('ebrasil', 'product_category_name_translation.csv'))
cat = pd.merge(products, products_translated, on="product_category_name", how='left')
# cat = cat.drop(columns="product_category_name")

In [5]:
products.shape,cat.shape

((32951, 9), (32951, 10))

In [6]:
cat.isna().sum()

product_id                         0
product_category_name            610
product_name_lenght              610
product_description_lenght       610
product_photos_qty               610
product_weight_g                   2
product_length_cm                  2
product_height_cm                  2
product_width_cm                   2
product_category_name_english    623
dtype: int64

In [7]:
donnees = pd.read_csv(os.path.join('ebrasil', 'olist_products_dataset.csv'))
donnees.rename(columns={col:col.replace('product_','') for col in donnees.columns[1:]},inplace=True)

In [8]:
donnees.head()

,product_id,category_name,name_lenght,description_lenght,photos_qty,weight_g,length_cm,height_cm,width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [9]:
categories = pd.read_csv(os.path.join('ebrasil', 'product_category_name_translation.csv'))
categories.rename(columns={col:col.replace('product_','') for col in categories.columns},inplace=True)
categories.head()

,category_name,category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


In [10]:
donnees = donnees.merge(categories,how='left',on='category_name')

In [11]:
donnees[donnees.category_name_english.isna()][['category_name',
                                               'category_name_english']].drop_duplicates()

,category_name,category_name_english
105,NaN,NaN
1628,pc_gamer,NaN
5821,portateis_cozinha_e_preparadores_de_alimentos,NaN


In [12]:
donnees.loc[donnees.category_name_english.isna() & 
                    (donnees.category_name == 'pc_gamer'),'category_name_english'] = 'pc_gamer'
donnees.category_name_english[donnees.category_name_english.isna() & 
                    (donnees.category_name == 'portateis_cozinha_e_preparadores_de_alimentos') ] = 'kitchenware_tools_and_gadget'

In [13]:
donnees.category_name_english.fillna('not documented',inplace=True)

In [14]:
donnees.drop(columns='category_name',inplace=True)
donnees.rename(columns={'category_name_english':'category_name'},inplace=True)

In [15]:
donnees.head()

,product_id,name_lenght,description_lenght,photos_qty,weight_g,length_cm,height_cm,width_cm,category_name
0,1e9e8ef04dbcff4541ed26657ea517e5,40.0,287.0,1.0,225.0,16.0,10.0,14.0,perfumery
1,3aa071139cb16b67ca9e5dea641aaa2f,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,art
2,96bd76ec8810374ed1b65e291975717f,46.0,250.0,1.0,154.0,18.0,9.0,15.0,sports_leisure
3,cef67bcfe19066a932b7673e239eb23d,27.0,261.0,1.0,371.0,26.0,4.0,26.0,baby
4,9dc1a7de274444849c219cff195d0b71,37.0,402.0,4.0,625.0,20.0,17.0,13.0,housewares


In [16]:
donnees.isna().sum()

product_id              0
name_lenght           610
description_lenght    610
photos_qty            610
weight_g                2
length_cm               2
height_cm               2
width_cm                2
category_name           0
dtype: int64

## Sauvegarde en parquet

In [17]:
donnees.to_parquet('./ecommerce/products.parquet',compression='gzip', engine='pyarrow')

In [18]:
donnees.shape

(32951, 9)

In [19]:
!dir ecommerce

 Le volume dans le lecteur C s’appelle Windows-SSD
 Le numéro de série du volume est 84DC-AF7F

 Répertoire de C:\Users\etien\Documents\CCI\_HUB\cours\2024-2025\big_data\_git\donnees\ecommerce

21/11/2024  09:25    <DIR>          .
20/11/2024  15:02    <DIR>          ..
21/11/2024  09:22         4 807 544 items.parquet
21/11/2024  09:22         9 567 921 orders.parquet
21/11/2024  09:23        10 022 487 orders_payments.parquet
21/11/2024  09:23        13 281 965 orders_payments_items.parquet
21/11/2024  09:22         2 651 184 payments.parquet
21/11/2024  09:22         2 470 995 paymentsI.parquet
21/11/2024  09:25           961 962 products.parquet
21/11/2024  09:23         7 549 835 reviews.parquet
21/11/2024  09:23         2 845 170 reviews_p.parquet
               9 fichier(s)       54 159 063 octets
               2 Rép(s)  708 369 448 960 octets libres
